___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [5]:
import requests

requests.get("https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange").status_code

200

2. Отримати курс долара за останній рік.

In [9]:
from datetime import date, timedelta
url = "https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json"
year_rates = []
today = date.today()

for i in range(365):
    new_date = today - timedelta(days=i)
    r = requests.get(url, params={'valcode' : 'USD', 'date' : new_date.strftime('%Y%m%d')}).json()
    year_rates.append(r[0])

3. Вивести середнє значення та відхилення курсу за кожний місяць.

_Cортування курсів по місяцях_

In [40]:
months_rates = {}

for rate in year_rates:
    month = rate['exchangedate'].split('.')[1]
    months_rates.setdefault(month, []).append(rate) 

_Cереднє значення за кожний місяць_

In [43]:
avarage_months_rates = {}

for month, rates in months_rates.items():
    rates_sum = sum([rate['rate'] for rate in rates])
    avarage_months_rates[month] = rates_sum / len(rates)
    
print("середнє значення за кожний місяць: ", avarage_months_rates)

середнє значення за кожний місяць:  {'08': 27.202158064516134, '07': 27.214548387096773, '06': 27.239873333333335, '05': 27.60284838709677, '04': 27.930006666666664, '03': 27.79555806451613, '02': 27.88466071428572, '01': 28.21977096774193, '12': 28.169719354838726, '11': 28.310153333333332, '10': 28.324951612903224, '09': 27.976716666666665}


_Відхилення курсу за кожний місяц_

In [42]:
months_deviations = {}

for month, month_rates in months_rates.items():
    result = 0
    for rate_data in month_rates:
        result += (rate_data['rate'] - avarage_month_rates[month]) ** 2

    months_deviations[month] = (result / len(month_rates)) ** 0.5

print("відхилення курсу за кожний місяць:", months_deviations)

відхилення курсу за кожний місяць: {'08': 0.28087748881903335, '07': 0.1532937636737697, '06': 0.15346138696391648, '05': 0.11849255828264614, '04': 0.08930549790217603, '03': 0.113636249822287, '02': 0.10439519125243588, '01': 0.12115058663084943, '12': 0.24570683871868976, '11': 0.14835098411837006, '10': 0.054615157242512376, '09': 0.23686450850409987}


4. Дану інформацію записати у файл за допомогою pickle.

In [39]:
import pickle

with open('rates.pickle', 'wb') as f:
    pickle.dump({'avarage month rates' : avarage_months_rates, 'months deviations' : months_deviations}, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

_Створення бази даних_

In [45]:
import sqlite3

shop_db = sqlite3.connect('shop.db')
curs = shop_db.cursor()
sql_req = """ 
    CREATE TABLE medications (
        sku INTEGER,
        name TEXT,
        price INTEGER
    )
 """
curs.execute(sql_req)
shop_db.commit()
shop_db.close()

_Заповнення бази даними_

In [47]:
from bs4 import BeautifulSoup
import requests


shop_db = sqlite3.connect('shop.db')
curs = shop_db.cursor()

# знаходимо к-сть сторінок
url = 'https://smallpacking.agrosem.ua/shop'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
pages = int(soup.find_all('a', class_='page-numbers')[-2].text)

# проходимо по всіх сторінках
for page in range(1, pages + 1):
    r = requests.get(f'{url}/page/{page}/')
    soup = BeautifulSoup(r.text, 'lxml')
    products_data = soup.find_all('div', class_="product-from-category-container")
    
    # проходимо по веб елементах кожного продукта
    for data in products_data:
        sku = data.find('a', class_='button product_type_simple add_to_cart_button ajax_add_to_cart')
        sku = int(sku.get('data-product_sku'))
        name = data.find('span', class_='pruduct-name-bottom').text
        price = data.find('span', class_='regular-price').text
        price = int(price.replace('грн', '').replace(' ', ''))        

        sql_req = f""" 
            INSERT INTO medications(sku, name, price)
            VALUES ({sku}, "{name}", {price})
        """
        curs.execute(sql_req)

shop_db.commit()
shop_db.close()

_Відображення даних_

In [55]:
con = sqlite3.connect("shop.db")
cursor = con.cursor()
cursor.execute("SELECT * FROM medications")
#print(*cursor.fetchall(), sep='\n')

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

2. Виведіть перелік всіх таблиць.

3. Список всіх cust_id з таблиці Customers table.

4. Всю таблицю Customers table

5. Список клієнтів (cust_names) відсортованих від Z до A

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

8. Товар (з таблиці Products), ціна якого становить 9.49

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

10. Кількість товару, що було продано

11. Кількість найменувань товару, ціна якого більша за 4

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди
